In [1]:
import cv2
from cv2 import aruco
from matplotlib import pyplot as plt
import math
from math import atan2, cos, sin, sqrt, pi
import numpy as np

In [2]:
def convertWheelSpeeds(speed_l,speed_r,speed_factor=3,axle_track=94):
    '''
    Function that takes the raw values of each motor angular speed and converts it into
    translation speed (in mm/s) and rotation speed (rad/sec) with a pixel scaling factor.

    Inputs: - speed_l: speed of left motor (no units, from Thymio measurements)
            - speed_r: speed of right motor (no units, from Thymio measurements)
            - speed_factor: 3 (motor_target of 200 sent to Thymio corresponds to a wheel speed of 66.8 mm/s)
            - axle_track: 94mm (distance between both wheels)

    Output: - v  : The current forward velocity of the Thymio (in mm/s)
            - omega : The current angular velocity of the Thymio (in rad/s)(positive direction corresponding to that of the camera frame)
    '''
    # wheel_radius : 22mm
    # axle track : 94mm (distance between the 2 wheels)
    v = 0.5*(speed_l + speed_r) / speed_factor
    omega = (speed_l - speed_r) / (0.5*axle_track*speed_factor)
    
    return v, omega
    

In [3]:
def px2real(x_px, scalingFactor):
    '''
    Function that converts measurement in pixels to measurement in mm.

    Inputs: - x_px : value in pixels
            - scalingFactor : for image size (1920x1080) corresponding to real length and width (800mm x 450mm), the scalingFactor is 2.4 

    Output: - x_real : value in mm
            
    '''
    x_real = x_px / 2.4
    return x_real
    
    

In [4]:
def real2px(x_real, scalingFactor):
    '''
    Function that converts measurement in mm to measurement in pixels.

    Inputs: - x_real : value in mm
            - scalingFactor : for image size (1920x1080) corresponding to real length and width (800mm x 450mm), the scalingFactor is 2.4 

    Output: - x_px : value in pixels
            
    '''
    x_px = x_real * 2.4
    return x_px
    

In [5]:
'''
Extended Kalman Filter class

Code modified from https://github.com/L42Project/Tutoriels/blob/master/Divers/tutoriel36/KalmanFilter.py
Comments describing Kalman Filter taken from https://stackoverflow.com/questions/74318200/how-to-tune-extended-kalman-filter-on-pykalman

Example of instance: KF=KalmanFilter(0.03333, [0, 0, 0, 0, 0]) (dt = 0.033 is the time between each measurement, x_ini = [0, 0, 0, 0, 0] is the initial state)

For update() function : takes as input the measurement vector z
z[0:2] are the measurements from the camera (px, py, orientation)
z[3:4] are the measurements from the Thymio (forward speed v and angular speed omega)

When Thymio position not found by camera -> z becomes 2x1 vector, with z[0:1] being the forward speed v and angular speed omega
'''


class KalmanFilter():
    def __init__(self, dt, x_ini):
        self.dt = dt

        # Initial State Vector
        self.x=np.matrix([[x_ini[0]], [x_ini[1]], [x_ini[2]], [x_ini[3]], [x_ini[4]]])
    

        # Initialize State Transition Matrix (Warning : changes with time -> updateFk())
        self.Fk=np.eye(5)

        # Initialize Measurement matrix H
        # Used to convert the predicted state estimate into predicted sensor measurements at time k.
        # In this case, H will be the identity matrix since the estimated state maps directly to state measurements data
        # H has the same number of rows as sensor measurements and same number of columns as states.
        self.H=np.eye(5)

        
        # State model noise covariance matrix Q
        # When Q is large, the Kalman Filter tracks large changes in
        # the sensor measurements more closely than for smaller Q.
        # Q is a square matrix that has the same number of rows as states.
        self.Q=np.diag([1, 0.1, 0.01, 1.0, 1.0]) 
        

        # Sensor measurement noise covariance matrix R
        # Has the same number of rows and columns as sensor measurements.
        # If we are sure about the measurements, R will be near zero.
        self.coord_var = 0.01
        self.angle_var = 0.0001
        self.speed_var = 6.0
        self.omega_var = 0
        
        self.R=np.diag([self.coord_var, self.coord_var, self.angle_var, self.speed_var, self.omega_var])

        self.P=np.eye(5)
        
    def updateFk(self):
        '''
        Function that updates the state transition matrix Fk
        '''
        
        self.Fk=np.matrix([[1.0, 0, 0, self.dt*math.cos(self.x[2]), 0],
                           [0, 1.0,  0, self.dt*math.sin(self.x[2]), 0],
                           [0, 0, 1.0, 0, self.dt],
                           [0, 0, 0, 1.0, 0],
                           [0, 0, 0, 0, 1.0]])
        self.Fk = self.Fk.astype(float)

    def predict(self):
        # Update Fk
        self.updateFk()
        # Predict the state estimate (A Priori) at time k based on the state estimate at time k-1
        self.x=np.dot(self.Fk, self.x)
        # Predict the state covariance estimate based on the previous covariance and some noise
        self.P=np.dot(np.dot(self.Fk, self.P), self.Fk.T)+self.Q
        return self.x

    def update(self, z, CameraAccessible = True): # z[0:2] corrpesonds to measurement of camera, z[3:4] corresponds to measurements of wheels
        
        if CameraAccessible:
            self.H=np.eye(5)
            
            self.R=np.diag([self.coord_var, self.coord_var, self.angle_var, self.speed_var, self.omega_var])
        else: # CAUTION : IF CAMERAACCESSIBLE = FALSE, Z IS A 2x1 VECTOR (z[0:1] are now measrurements from wheels)
            # Measurement matrix H (now only v and omega can be observed)
            self.H=np.matrix([[0, 0, 0, 1, 0],
                              [0, 0, 0, 0, 1]])
            
            self.R=np.diag([self.speed_var, self.omega_var])
            
            
        # Compute Kalman gain
        S=np.dot(self.H, np.dot(self.P, self.H.T))+self.R
        inv_S = np.linalg.pinv(S.astype(float))
        K=np.dot(np.dot(self.P, self.H.T),inv_S)

        # Correction / innovation
        # Calculate an updated state estimate (A Posteriori) for time k
        self.x=self.x+np.dot(K, (z-np.dot(self.H, self.x)))
        # Update the state covariance estimate for time k
        I=np.eye(self.H.shape[1])
        self.P=(I-(K*self.H))*self.P


        return self.x

In [17]:
def dist(pt1, pt2):
    distance = ( (pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2 )**0.5
    return distance



def get_thymio_position(img, img_output):
    
    '''
    Function that outputs Thymio position, radius and orientation from camera frames using Aruco marker detection
    Parts of code taken from https://mecaruco2.readthedocs.io/en/latest/notebooks_rst/Aruco/aruco_basics.html

    Inputs: - img : image from camera in BGR format
            - img_output : copy of image img on which will be drawn Thymio's contours

    Output: - center : tuple (x,y) representing the center of the Thymio (in pixels)
            - radius : radius of the Thymio (in pixels)
            - angle : orientation of the Thymio (in radians)
            - thymio_detected : boolean set to True if Thymio was detected, False if not
    '''
    
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
    parameters =  aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    frame_markers = aruco.drawDetectedMarkers(img_output, corners, ids)
    
    
    thymio_detected = False
    center = (0,0)
    radius = 0
    angle = 0
    
    if np.all(ids != None):
        for i in range(len(ids)):
            if ids[i] == 0:
                thymio_detected = True
                c = corners[i][0]

                top_center = (c[0:2, 0].mean(),c[0:2, 1].mean())
                bottom_center = (c[2:4, 0].mean(), c[2:4, 1].mean())
                left_center = (c[0:4:3, 0].mean(), c[0:4:3, 1].mean())
                right_center = (c[1:3, 0].mean(), c[1:3, 1].mean())
                points = np.array([ [c[2,0],c[2,1]], [c[3,0],c[3,1]], [left_center[0],left_center[1]], [right_center[0],right_center[1]] ])

                square_center = (int(c[:, 0].mean()), int(c[:, 1].mean()))
                radius = int(max( dist(square_center,c[0]), dist(square_center,c[1]), dist(square_center,c[2]), dist(square_center,c[3])))
                angle = math.atan2(top_center[1]-bottom_center[1],top_center[0]-bottom_center[0])
                
                
                center = (int(points[:, 0].mean()), int(points[:, 1].mean()))
                cv2.arrowedLine(img_output, (center[0],center[1]), (int(top_center[0]),int(top_center[1])), (255,255,0), 10)
                cv2.circle(img_output,square_center,radius,(255,0,0),10)

                break

        
    return center, radius, angle, thymio_detected

In [18]:
'''
Testing Kalman Filter
'''

global center, radius, angle, thymio_detected

forward_speed = 300
angular_speed = 0

KF=KalmanFilter(0.033, [500, 300, math.pi/4, forward_speed, angular_speed])



### INITIALIZE CAMERA
cap = cv2.VideoCapture(0)
fps = int(cap.get(5))
print("fps:", fps)



### MAIN LOOP
count = 0

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# running infinite while loop so that
# program keep running until we close it
while True:
    success, img = cap.read()
    imgResult = img.copy()
    
    thymioLoc, thymioRadius, thymioAngle, thymioDetected = get_thymio_position(img, imgResult)
    
    # KALMAN -------------------------------------------------------------------
    KF.predict()
    if thymioDetected:
        z = np.array([thymioLoc[0],thymioLoc[1],thymioAngle, forward_speed, angular_speed])
        z = np.expand_dims(z, axis=-1)
    else:
        z = np.array([forward_speed, angular_speed]) 
        z = np.expand_dims(z, axis=-1)
    KF.update(z,thymioDetected)
    cv2.circle(imgResult, (int(KF.x[0]), int(KF.x[1])), 10, (255,0,255), 10)
    # KALMAN ----------------------------------------------------------------------
    
    count = count + 1
    print(count)
    print('Thymio center:',thymioLoc, '  Angle:',thymioAngle, '  Thymio Detected:' ,thymioDetected)
    print('Kalman center:',KF.x[0:2],'  Kalman angle:',KF.x[2], 'Kalman speed and omega', KF.x[3:5])
 

    # displaying output on Screen
    cv2.imshow("Result", imgResult)
    
    c = cv2.waitKey(1)
    if c == 27:
        break

fps: 1
1
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[507.00035713]
 [307.00035713]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
2
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[514.00071427]
 [314.00071427]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
3
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[521.0010714]
 [321.0010714]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
4
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[528.00142853]
 [328.00142853]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
5
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[535.00178567]
 [335.00178567]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
6
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[542.0021428]
 [342.0021

49
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[843.01749955]
 [643.01749955]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
50
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[850.01785669]
 [650.01785669]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
51
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[857.01821382]
 [657.01821382]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
52
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[864.01857095]
 [664.01857095]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
53
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[871.01892809]
 [671.01892809]]   Kalman angle: [[0.78539816]] Kalman speed and omega [[300.]
 [  0.]]
54
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[878.01928522]
 [678.01

89
Thymio center: (320, 336)   Angle: 0.2868198185830905   Thymio Detected: True
Kalman center: [[320.16082495]
 [335.02307079]]   Kalman angle: [[0.28753505]] Kalman speed and omega [[300.68537623]
 [  0.        ]]
90
Thymio center: (313, 343)   Angle: 0.24425455029180246   Thymio Detected: True
Kalman center: [[313.16341819]
 [342.56648474]]   Kalman angle: [[0.24467891]] Kalman speed and omega [[300.34436764]
 [  0.        ]]
91
Thymio center: (308, 352)   Angle: 0.2014962984336475   Thymio Detected: True
Kalman center: [[308.14490628]
 [351.40981743]]   Kalman angle: [[0.2019197]] Kalman speed and omega [[300.28018901]
 [  0.        ]]
92
Thymio center: (294, 368)   Angle: 0.11338026425924755   Thymio Detected: True
Kalman center: [[294.23390778]
 [366.77458964]]   Kalman angle: [[0.11424838]] Kalman speed and omega [[300.33110615]
 [  0.        ]]
93
Thymio center: (293, 376)   Angle: 0.07006058263845544   Thymio Detected: True
Kalman center: [[293.10857835]
 [375.32055304]]   Kal

129
Thymio center: (425, 420)   Angle: -0.42722910537531444   Thymio Detected: True
Kalman center: [[425.06712366]
 [418.81907147]]   Kalman angle: [[-0.42747428]] Kalman speed and omega [[293.8253432]
 [  0.       ]]
130
Thymio center: (425, 428)   Angle: -0.4298163331506899   Thymio Detected: True
Kalman center: [[425.08671969]
 [426.89406885]]   Kalman angle: [[-0.42979337]] Kalman speed and omega [[293.39541203]
 [  0.        ]]
131
Thymio center: (427, 427)   Angle: -0.4144545924705449   Thymio Detected: True
Kalman center: [[427.06764463]
 [426.65210933]]   Kalman angle: [[-0.41460499]] Kalman speed and omega [[294.64816106]
 [  0.        ]]
132
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[436.00326147]
 [422.71853152]]   Kalman angle: [[-0.41460499]] Kalman speed and omega [[296.42875317]
 [  0.        ]]
133
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[444.99659606]
 [418.76002836]]   Kalman angle: [[-0.41460499]] Kalm

174
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[549.8649594 ]
 [427.73449824]]   Kalman angle: [[-0.32415778]] Kalman speed and omega [[299.49522252]
 [  0.        ]]
175
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[559.24317356]
 [424.58331472]]   Kalman angle: [[-0.32415778]] Kalman speed and omega [[299.6634774]
 [  0.       ]]
176
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[568.6236553 ]
 [421.43137379]]   Kalman angle: [[-0.32415778]] Kalman speed and omega [[299.77565034]
 [  0.        ]]
177
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[578.00553476]
 [418.27896521]]   Kalman angle: [[-0.32415778]] Kalman speed and omega [[299.85043318]
 [  0.        ]]
178
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[587.38829537]
 [415.12626144]]   Kalman angle: [[-0.32415778]] Kalman speed and omega [[299.90028868]
 [  0.        ]]
179
Thymio c

...

2254
Thymio center: (557, 353)   Angle: 2.836467645592829   Thymio Detected: True
Kalman center: [[556.89768622]
 [353.3587301 ]]   Kalman angle: [[2.8364915]] Kalman speed and omega [[297.60310611]
 [  0.        ]]
2255
Thymio center: (556, 357)   Angle: 2.843093722003614   Thymio Detected: True
Kalman center: [[555.91688882]
 [356.94218779]]   Kalman angle: [[2.84302899]] Kalman speed and omega [[298.13143311]
 [  0.        ]]
2256
Thymio center: (556, 359)   Angle: 2.850858402338788   Thymio Detected: True
Kalman center: [[555.90703755]
 [359.07003187]]   Kalman angle: [[2.85078164]] Kalman speed and omega [[298.26499374]
 [  0.        ]]
2257
Thymio center: (555, 356)   Angle: 2.8424035909882654   Thymio Detected: True
Kalman center: [[554.91668719]
 [356.49380505]]   Kalman angle: [[2.84248574]] Kalman speed and omega [[297.8306223]
 [  0.       ]]
2258
Thymio center: (555, 353)   Angle: 2.83949412520103   Thymio Detected: True
Kalman center: [[554.90735403]
 [353.53552406]]   Kal

2295
Thymio center: (550, 362)   Angle: 2.8436616617409314   Thymio Detected: True
Kalman center: [[549.90711419]
 [362.51647471]]   Kalman angle: [[2.8437327]] Kalman speed and omega [[297.44175674]
 [  0.        ]]
2296
Thymio center: (550, 360)   Angle: 2.833960252564795   Thymio Detected: True
Kalman center: [[549.90727634]
 [360.45240052]]   Kalman angle: [[2.83405607]] Kalman speed and omega [[297.28521076]
 [  0.        ]]
2297
Thymio center: (548, 356)   Angle: 2.813039483038342   Thymio Detected: True
Kalman center: [[547.92726462]
 [356.62217668]]   Kalman angle: [[2.81324555]] Kalman speed and omega [[297.0085024]
 [  0.       ]]
2298
Thymio center: (548, 356)   Angle: 2.8142624918752634   Thymio Detected: True
Kalman center: [[547.90842439]
 [356.3171417 ]]   Kalman angle: [[2.81425252]] Kalman speed and omega [[297.14749209]
 [  0.        ]]
2299
Thymio center: (546, 357)   Angle: 2.813592180571452   Thymio Detected: True
Kalman center: [[545.92769716]
 [357.20720278]]   K

2335
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[336.67615463]
 [416.4527086 ]]   Kalman angle: [[2.89913105]] Kalman speed and omega [[299.99860818]
 [  0.        ]]
2336
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[327.06574547]
 [418.82962512]]   Kalman angle: [[2.89913105]] Kalman speed and omega [[299.99907212]
 [  0.        ]]
2337
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[317.45533134]
 [421.20654287]]   Kalman angle: [[2.89913105]] Kalman speed and omega [[299.99938141]
 [  0.        ]]
2338
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[307.84491391]
 [423.58346144]]   Kalman angle: [[2.89913105]] Kalman speed and omega [[299.99958761]
 [  0.        ]]
2339
Thymio center: (463, 375)   Angle: 2.804458697412556   Thymio Detected: True
Kalman center: [[462.92238107]
 [375.23859054]]   Kalman angle: [[2.80450374]] Kalman speed and omega [[298.24932292]
 [  0.  

2376
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[536.9518943 ]
 [382.35915949]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.02641966]
 [  0.        ]]
2377
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[538.02725839]
 [392.20087608]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.01761311]
 [  0.        ]]
2378
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[539.10261184]
 [402.04249528]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.01174207]
 [  0.        ]]
2379
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[540.17795822]
 [411.8840498 ]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.00782805]
 [  0.        ]]
2380
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[541.2532999]
 [421.7255613]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.0052187]
 [  0.       ]]
2381
Thymio ce

2420
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[584.26661081]
 [815.38276191]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.]
 [  0.]]
2421
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[585.34194312]
 [825.22418764]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.]
 [  0.]]
2422
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[586.41727542]
 [835.06561337]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.]
 [  0.]]
2423
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[587.49260773]
 [844.90703911]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.]
 [  0.]]
2424
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[588.56794003]
 [854.74846484]]   Kalman angle: [[1.46196217]] Kalman speed and omega [[300.]
 [  0.]]
2425
Thymio center: (0, 0)   Angle: 0   Thymio Detected: False
Kalman center: [[589.643272